### Importing all the essential libraries

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time

#### Sample Username and Password

In [2]:
u_name = 'Sample_Username'
p_word = 'Sample_Password'

In [5]:
def log_in_function(driver, username, password):
    ### Getiing both the input fields
    time.sleep(2)
    input_field = driver.find_elements('tag name', 'input')
    u_name_field = input_field[0]
    p_field = input_field[1]

    ### Entering Username
    u_name_field.send_keys(username)
    
    ### Entering Password
    p_field.send_keys(password)
    
    ### Clicking on log in button... Note: This can also be done by simply sending 'Keys.ENTER' into p_field
    log_btn = driver.find_element('xpath', '/html/body/div[2]/div/div/div[2]/div/div/div[1]/section/main/article/div[2]/div[1]/div[2]/form/div/div[3]/button')
    log_btn.click()

    ### Wait for some time to log in and then proceed to the next task
    time.sleep(4)

In [4]:
def search_bar(driver, word):
    ### Selecting the search element
    driver.get('https://www.instagram.com/accounts/onetap/?next=%2F')
    search = driver.find_element('xpath', '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/div/div/div/div/div[2]/div[2]/span/div/a/div/div[1]/div/div')
    search.click()                  

    ### Selecting the input box and sending the word to search
    inp = driver.find_element('tag name', 'input')
    inp.send_keys(word)

    ### Wait for few seconds to load search results before proceeding
    time.sleep(2)

In [5]:
def clear_search_bar(driver):
    ### Selecting and clicking on the search element again to reset it
    search = driver.find_element('xpath', '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/div/div/div/div/div[2]/div[2]/span/div/a/div/div[1]/div/div')
    search.click()
    
    time.sleep(1)

In [4]:
def extract_handles(driver):
    ### Finding all the handles_div in the search result
    handles = driver.find_elements(By.CSS_SELECTOR, 'span.x1lliihq.x1plvlek.xryxfnj.x1n2onr6.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.x1i0vuye.xvs91rp.x1s688f.x5n08af.x10wh9bi.x1wdrske.x8viiok.x18hxmgj')

    handle_list = []
    for handle in handles:
        ### Skipping all hastags results
        if handle.text.startswith('#') or ' ' in handle.text:
            continue
        handle_list.append(handle.text)
    
    return handle_list

In [3]:
def open_profile(driver, profile_name):
    ### Searching the given profile
    search_bar(driver, profile_name)
    
    ### Selecting the top profile and clicking on it
    top_handle = driver.find_element(By.CSS_SELECTOR, 'span.x1lliihq.x1plvlek.xryxfnj.x1n2onr6.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.x1i0vuye.xvs91rp.x1s688f.x5n08af.x10wh9bi.x1wdrske.x8viiok.x18hxmgj')
    top_handle.click()

In [8]:
def follow(driver, profile_name):
    ### Opening the profile to follow
    driver.get('https://www.instagram.com/accounts/onetap/?next=%2F')
    open_profile(driver, profile_name)
    
    time.sleep(2)

    ### Selecting the follow button parent div
    follow_div = driver.find_element(By.CSS_SELECTOR, 'div._ap3a._aaco._aacw._aad6._aade')
    
    ### Checking if we already follow this profile
    if follow_div.text == 'Following':
        return 0

    ### Selecting and clicking on the follow button
    follow_button = driver.find_element(By.CSS_SELECTOR, 'button._acan._acap._acas._aj1-._ap30')
    follow_button.click()
    return 1

In [9]:
def unfollow(driver, profile_name):
    ### Opening the profile to unfollow
    driver.get('https://www.instagram.com/accounts/onetap/?next=%2F')
    open_profile(driver, profile_name)
    
    time.sleep(2)

    ### Selecting the follow button parent div
    follow_div = driver.find_element(By.CSS_SELECTOR, 'div._ap3a._aaco._aacw._aad6._aade')
    
    ### Checking if we follow the profile or not
    if follow_div.text == 'Follow':
        return 0

    ### selecting the unfollow button and clicking on it, it will open a dialog box
    follow_button = driver.find_element(By.CSS_SELECTOR, 'button._acan._acap._acat._aj1-._ap30')
    follow_button.click()

    time.sleep(1.5)

    ### Selecting and clicking the unfollow button from the opened dialog box
    unfollow_button = driver.find_elements(By.CSS_SELECTOR, 'div.x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1iyjqo2.x2lwn1j.xeuugli.xdt5ytf.xqjyukv.x1cy8zhl.x1oa3qoh.x1nhvcw1')[4]
    unfollow_button.click()
    return 1

In [10]:
def recent_posts(driver, profile_name):
    ### Opening the given profile
    driver.get('https://www.instagram.com/accounts/onetap/?next=%2F')
    open_profile(driver, profile_name)

    time.sleep(2)

    links = []
    count = 0
    current_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        ### Finding all the divs with the posts
        tot_divs = driver.find_elements(By.CSS_SELECTOR, 'div._ac7v.xras4av.xgc1b0m.xat24cr.xzboxd6')
        if len(tot_divs) < (10):
            ### Scrolling to end of current height
            driver.execute_script(f'window.scrollTo(0, {current_height})')
            time.sleep(1)
            new_height = driver.execute_script('return document.body.scrollHeight')
            
            ### Break loop, if no scroll happens
            if new_height != current_height:
                current_height = new_height
                continue
        else:
            break

    ### Extracting all the post links
    for div_element in tot_divs:
        divs = div_element.find_elements(By.CSS_SELECTOR, 'div.x1lliihq.x1n2onr6.xh8yej3.x4gyw5p.xfllauq.xo2y696.x11i5rnm.x2pgyrj')
        for each_div in divs:
            link = each_div.find_element('tag name', 'a')
            url = link.get_attribute('href')
            links.append(url)
            count += 1
            if count == 30:
                break
        if count == 30:
            break

    return links

In [11]:
def like_posts(driver, profile_name):
    ### Getting the list of links of the posts to like
    url_list = recent_posts(driver, profile_name)
    
    ### Print the list of post_urls to like
    urls = pd.DataFrame(url_list, columns = ['Post_URL to like'])
    urls['profile'] = profile_name
    print(urls)
    
    counter = 0
    for url in url_list:
        counter += 1

        ### Getting the post
        driver.get(url)
        time.sleep(1)
        
        ### Finding the like button parent div and the like button
        like_btn_div = driver.find_element(By.CSS_SELECTOR, 'div.x1i10hfl.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.x6s0dn4.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x1ypdohk.x78zum5.xl56j7k.x1y1aw1k.x1sxyh0.xwib8y2.xurb0ha.xcdnw81')
        like_btn = like_btn_div.find_element('tag name', 'svg')
        
        ### Checking if the post is already liked
        if like_btn.get_attribute('aria-label') == 'Unlike':
            print(f'Already liked this post - post number #{counter}')
            continue

        ### Clicking on the like button twice to like the post
        like_btn.click()
        time.sleep(0.5)
        like_btn_div = driver.find_element(By.CSS_SELECTOR, 'div.x1i10hfl.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.x6s0dn4.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x1ypdohk.x78zum5.xl56j7k.x1y1aw1k.x1sxyh0.xwib8y2.xurb0ha.xcdnw81')
        like_btn = like_btn_div.find_element('tag name', 'svg')
        like_btn.click()
        time.sleep(0.5)

In [12]:
def unlike_posts(driver, profile_name):
    ### Getting the list of links of the posts to unlike
    url_list = recent_posts(driver, profile_name)

    ### Print post_urls to unlike
    urls = pd.DataFrame(url_list, columns = ['Post_URL to un-like'])
    urls['profile'] = profile_name
    print(urls)
    
    counter = 0
    for url in url_list:
        counter += 1

        ### Getting the post
        driver.get(url)
        time.sleep(1)

        ### Finding the like button parent div and the like button
        like_btn_div = driver.find_element(By.CSS_SELECTOR, 'div.x1i10hfl.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.x6s0dn4.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x1ypdohk.x78zum5.xl56j7k.x1y1aw1k.x1sxyh0.xwib8y2.xurb0ha.xcdnw81')
        like_btn = like_btn_div.find_element('tag name', 'svg')

        ### Checking if the post is already liked or not
        if like_btn.get_attribute('aria-label') == 'Like':
            print(f'Not liked initially - post number #{counter}')
            continue

        ### Clicking on the like button to unlike the post
        like_btn.click()
        time.sleep(0.5)

In [13]:
### If number is not provided, it will extract all the follower user_names.
def extract_followers(driver, profile_name, number = 99999999999999):
    ### Opening the given profile
    driver.get('https://www.instagram.com/accounts/onetap/?next=%2F')
    open_profile(driver, profile_name)
    time.sleep(2)

    ### Finding followers button and clicking on it to open followers dialog box
    followers_btn = driver.find_element(By.CSS_SELECTOR, 'a.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xt0psk2.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz._alvs._a6hd')
    followers_btn.click()
    time.sleep(2)

    follower_divs = []
    p_len = -1
    prev_len = -1
    while True:
        ### Finding all the divs, containing follower information
        follower_div = driver.find_elements(By.CSS_SELECTOR, 'div.x1dm5mii.x16mil14.xiojian.x1yutycm.x1lliihq.x193iq5w.xh8yej3')
        # print(follower_div)

        ### Adding all the elements in a list
        follower_divs.extend(follower_div)

        ### Removing duplicate elements from the list
        fset = set(follower_divs)
        follower_divs = list(fset)
        
        ### End scrolling and continue with available data, if end of follower list is reached.
        try:
            ### Selecting Scroll element
            scroller = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div/div/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[3]/div[2]/div/div')
            scroller.click()
        except:
            break

        ### Implementing Scroll Action, with appropriate time gaps.
        actions = webdriver.ActionChains(driver)
        actions.move_to_element(scroller).perform()
        actions.send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(2)
        
        ### Breaking the loop, if the number is reached
        if (len(follower_divs)) >= number:
            break

        ### Breaking the loop, if the length of the list doesnot change, that is, if scroll action doesn't happen
        curr_len = len(follower_divs)
        if p_len == curr_len:
            break
        p_len = prev_len
        prev_len = curr_len
            
    ### Extracting followers usernames
    followers_list = []
    for follower in follower_divs:
        if number == 0:
            break
        number -= 1
        user_field = follower.find_element(By.TAG_NAME, 'a')
        follower_user_name = user_field.get_attribute('href')[26:-1]
        followers_list.append(follower_user_name)

    return followers_list

In [14]:
### If number is not provided, it will extract all the following user_names.
def extract_following(driver, profile_name, number = 99999999999999):
    ### Opening the given profile
    driver.get('https://www.instagram.com/accounts/onetap/?next=%2F')
    open_profile(driver, profile_name)
    time.sleep(2)

    ### Finding following button and clicking on it to open following dialog box
    following_btn = driver.find_elements(By.CSS_SELECTOR, 'a.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xt0psk2.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz._alvs._a6hd')[1]
    following_btn.click()
    time.sleep(1.5)

    following_divs = []
    p_len = -1
    prev_len = -1
    while True:
        ### Finding all the divs, containing following information
        following_div = driver.find_element('xpath', '/html/body/div[6]/div[2]/div/div/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[4]/div[1]/div').find_elements(By.CSS_SELECTOR, 'div.x1dm5mii.x16mil14.xiojian.x1yutycm.x1lliihq.x193iq5w.xh8yej3')
        ### Adding all the elements in a list
        following_divs.extend(following_div)

        ### Removing duplicate elements from the list
        fset = set(following_divs)
        following_divs = list(fset)
        # print(len(following_divs))
        
        ### End scrolling and continue with available data, if end of follower list is reached.
        try:
            ### Selecting Scroll element
            scroller = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div/div/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[4]/div[2]')
            scroller.click()
        except:
            break

        ### Implementing Scroll Action, with appropriate time gaps.
        actions = webdriver.ActionChains(driver)
        actions.move_to_element(scroller).perform()
        actions.send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(2)
        
        ### Breaking the loop, if the number is reached
        if (len(following_divs)) >= number:
            break
        
        ### Breaking the loop, if the length of the list doesnot change, that is, if scroll action doesn't happen
        curr_len = len(following_divs)
        if p_len == curr_len:
            break
        p_len = prev_len
        prev_len = curr_len
            
    ### Extracting followers usernames
    followings_list = []
    for following in following_divs:
        if number == 0:
            break
        number -= 1
        user_field = following.find_element(By.TAG_NAME, 'a')
        following_user_name = user_field.get_attribute('href')[26:-1]
        followings_list.append(following_user_name)

    return followings_list

In [15]:
def view_story(driver, given_user):
    ### Following the profile and save status to unfollow later, if were not following earlier
    follow_status = follow(driver, given_user)
    time.sleep(2)
    
    '''
    Open the profile of the user... 
    This is done to improve efficiency,
    that is, to simply skip further steps, if no story is uploaded.
    '''    
    try:
        ### Checking if story is uploaded
        canvas = WebDriverWait(driver, 5).until(EC.visibility_of(driver.find_element(By.CSS_SELECTOR, 'canvas.x1upo8f9.xpdipgo.x87ps6o')))
        # print('user has story')

        ### Extracting User Name
        username = driver.current_url[26:-1]

        ### Opening own profile page
        driver.get('https://www.instagram.com/')
        try:
            ### Extracting the div containing all the story elements
            story_div = WebDriverWait(driver, 5).until(EC.visibility_of(driver.find_element(By.CSS_SELECTOR, 'div._aac4._aac5._aac6._aj3f._ajdu')))
            
            ### Extracting all the story elements in a list
            stories = story_div.find_elements(By.CSS_SELECTOR, 'li._acaz')
            for story in stories:
                ### Extracting the user name
                user = story.find_element(By.CSS_SELECTOR, 'div.x9f619.x1lliihq.x6ikm8r.x10wlt62.x1n2onr6.x2b8uid.xlyipyv.xuxw1ft.x1yf5rgg.xhikscq.xg83lxy.x1h0ha7o').text

                ### Comparing the given user name with the extracted username
                if user == username:

                    ### Extracting the story button
                    btn = story.find_element('tag name', 'button')

                    ### Checking the seen status of the story
                    seen_status = btn.get_attribute('aria-label').split(',')[1].strip()

                    if seen_status == 'seen':
                        print("Seen this story")
                        break
                    else:
                        ### Viewing the story, if not seen
                        btn.click()
                        time.sleep(5)
    
                        ### Returning to home page after a delay to prevent watching further stories
                        driver.get('https://www.instagram.com/')
                        
        except:
            print('Error')
    except:
        print('user has no story')

    ### Unfollow the profile, if not following earlier
    if follow_status:
        unfollow(driver, given_user)